# Concurrency in `rs` vs `py` for the Transformers Vectorization Microservice

Make sure to spin up the docker compose stack where the services are running with `docker compose up --build`

## Define the methods of the run

In [19]:
import concurrent.futures
import random
import requests
import string
import time
import typing

def send(s: requests.Session, port: int) -> float:
    start = time.time()
    txt = ''.join(random.choices(string.ascii_lowercase + string.digits, k=2560))
    res = s.post(f"http://localhost:{port}/vectors", json={
        "text": txt,
    })
    taken = time.time() - start
    if res.status_code != 200:
        print(f"Failed to retrieve vector with text {txt}")
        print(res.text)
    return taken


def load(port: int) -> None:
    session = requests.Session()
    for i in range(100):
        taken = send(session, port)
        print(f"Request {i} took {taken:.2f} seconds")

def run(port: int) -> None:
    futures: typing.List[concurrent.futures.Future] = []
    print("Starting benchmark")
    n_concurrent = 20
    start = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_concurrent) as executor:
        for _ in range(n_concurrent):
            futures.append(executor.submit(load, port))
        print("Scheduled all tasks, waiting for completion")
        concurrent.futures.wait(futures)
    print(f"Vectorised {n_concurrent * 100} texts in {time.time() - start:.2f} seconds")

## Run against the rust service

In [20]:
run(3000)

Starting benchmark
Request 0 took 0.01 seconds
Request 0 took 0.02 seconds
Scheduled all tasks, waiting for completion
Request 0 took 0.02 seconds
Request 0 took 0.02 seconds
Request 1 took 0.02 seconds
Request 0 took 0.05 seconds
Request 0 took 0.03 seconds
Request 2 took 0.01 seconds
Request 0 took 0.05 seconds
Request 1 took 0.01 seconds
Request 1 took 0.01 seconds
Request 1 took 0.04 seconds
Request 1 took 0.02 seconds
Request 3 took 0.02 seconds
Request 2 took 0.02 seconds
Request 4 took 0.01 seconds
Request 0 took 0.08 seconds
Request 3 took 0.01 seconds
Request 2 took 0.02 seconds
Request 2 took 0.03 seconds
Request 1 took 0.09 seconds
Request 0 took 0.09 seconds
Request 4 took 0.02 seconds
Request 3 took 0.02 seconds
Request 2 took 0.02 seconds
Request 1 took 0.01 seconds
Request 1 took 0.03 seconds
Request 5 took 0.01 seconds
Request 5 took 0.04 seconds
Request 4 took 0.01 seconds
Request 3 took 0.04 seconds
Request 2 took 0.01 seconds
Request 6 took 0.01 seconds
Request 2 too

## Run against the python service

In [21]:
run(8080)

Starting benchmark
Scheduled all tasks, waiting for completion
Request 0 took 0.03 seconds
Request 0 took 0.03 seconds
Request 0 took 0.03 seconds
Request 0 took 0.04 seconds
Request 0 took 0.05 seconds
Request 0 took 0.04 seconds
Request 0 took 0.04 seconds
Request 0 took 0.05 seconds
Request 0 took 0.07 seconds
Request 0 took 0.08 seconds
Request 0 took 0.07 seconds
Request 0 took 0.09 seconds
Request 0 took 0.09 seconds
Request 1 took 0.06 seconds
Request 1 took 0.06 seconds
Request 1 took 0.06 seconds
Request 0 took 0.08 seconds
Request 0 took 0.09 seconds
Request 0 took 0.09 seconds
Request 0 took 0.09 seconds
Request 1 took 0.07 seconds
Request 1 took 0.07 seconds
Request 0 took 0.09 seconds
Request 0 took 0.10 seconds
Request 0 took 0.11 seconds
Request 1 took 0.07 seconds
Request 1 took 0.07 seconds
Request 1 took 0.06 seconds
Request 1 took 0.06 seconds
Request 1 took 0.04 seconds
Request 1 took 0.05 seconds
Request 1 took 0.04 seconds
Request 1 took 0.05 seconds
Request 2 too